In [1]:
# import modules

import os
import pandas as pd

In [2]:
# set working path and import BUSCO TSV files

abspath = os.path.abspath('')
os.chdir(abspath)

ref_genome = pd.read_csv('./reference_genome_full_table.tsv', sep='\t', skiprows=2)
ref_proteome = pd.read_csv('./reference_proteome_full_table.tsv', sep='\t', skiprows=2)
de_novo = pd.read_csv('./trinity_full_table.tsv', sep='\t', skiprows=2)

In [3]:
# replace spaces with underscores in headers, rename busco id column

ref_genome.columns = ref_genome.columns.str.replace(' ', '_')
ref_proteome.columns = ref_proteome.columns.str.replace(' ', '_')
de_novo.columns = de_novo.columns.str.replace(' ', '_')

ref_genome.rename(columns={'#_Busco_id':'Busco_id'}, inplace=True)
ref_proteome.rename(columns={'#_Busco_id':'Busco_id'}, inplace=True)
de_novo.rename(columns={'#_Busco_id':'Busco_id'}, inplace=True)

In [9]:
# retain first duplicated row, discard the rest

ref_genome.drop_duplicates(subset = 'Busco_id', keep = 'first', inplace = True)
ref_proteome.drop_duplicates(subset = 'Busco_id', keep = 'first', inplace = True)
de_novo.drop_duplicates(subset = 'Busco_id', keep = 'first', inplace = True)

In [8]:
# create dataframes with complete/duplicated BUSCO IDs

# ref_genome complete sequences only
# ref_genome_complete = ref_genome[(ref_genome['Status'] == 'Complete')]
ref_genome_complete = ref_genome[(ref_genome['Status'].str.contains('Complete|Duplicated|Fragmented'))]
ref_proteome_complete = ref_proteome[(ref_proteome['Status'].str.contains('Complete|Duplicated|Fragmented'))]
de_novo_complete = de_novo[(de_novo['Status'].str.contains('Complete|Duplicated|Fragmented'))]

In [6]:
# identify complete de_novo BUSCO ids not in ref_genome or ref_proteome

de_novo_missing_from_ref_genome = de_novo_complete[~de_novo_complete.Busco_id.isin(ref_genome_complete.Busco_id)]
de_novo_missing_from_ref_proteome = de_novo_complete[~de_novo_complete.Busco_id.isin(ref_proteome_complete.Busco_id)]
print(f"De novo BUSCOs not in reference genome: {len(de_novo_missing_from_ref_genome)}")
print(f"De novo BUSCOs not in reference proteome: {len(de_novo_missing_from_ref_proteome)}")

De novo BUSCOs not in reference genome: 193
De novo BUSCOs not in reference proteome: 247


In [7]:
# identify complete ref_genome and ref_proteome BUSCO ids not in de_novo

ref_genome_missing_from_de_novo = ref_genome_complete[~ref_genome_complete.Busco_id.isin(de_novo_complete.Busco_id)]
ref_proteome_missing_from_de_novo = ref_proteome_complete[~ref_proteome_complete.Busco_id.isin(de_novo_complete.Busco_id)]
print(f"Reference genome BUSCOs not in de novo transcriptome: {len(ref_genome_missing_from_de_novo)}")
print(f"Reference genome BUSCOs not in de novo transcriptome: {len(ref_proteome_missing_from_de_novo)}")

Reference genome BUSCOs not in de novo transcriptome: 220
Reference genome BUSCOs not in de novo transcriptome: 231


## BUSCO troubleshooting

In [1]:
from Bio import SeqIO

In [8]:
trinity_dict = {}
with open("/Users/danielli/Syncthing/UoB/unsynced/research_project/summarise_blast/trinity_full_table.tsv") as trinity_file:
    # create missing counter; missing plus trinity_dict should equal 2510, the number of BUSCOs in the hemiptera_odb10.2019-11-20 database
    trinity_missing_count = 0
    # skip first 3 header lines
    for _ in range(3):
        next(trinity_file)
    for line in trinity_file:
        # skip missing BUSCOs
        if line.split()[1] == 'Missing':
            trinity_missing_count += 1
            continue
        (buscoid, status, contigid, rest_of_line) = line.split('\t', 3)
        # get contig name by removing ':number-number' from contigid
        contig_name = contigid.split(':')[0]
        # if contig_name in trinity_dict:
        #     trinity_dict[contig_name] = f"{trinity_dict[contig_name]},{buscoid}"
        # else:
        #     trinity_dict[contig_name] = buscoid
        if buscoid in trinity_dict:
            trinity_dict[buscoid] = f"{trinity_dict[buscoid]},{contig_name}"
        else:
            trinity_dict[buscoid] = contig_name

In [10]:
refgen_dict = {}
with open("/Users/danielli/Syncthing/UoB/unsynced/research_project/summarise_blast/trinity_full_table.tsv") as refgen_file:
    # create missing counter; missing plus trinity_dict should equal 2510, the number of BUSCOs in the hemiptera_odb10.2019-11-20 database
    refgen_missing_count = 0
    # skip first 3 header lines
    for _ in range(3):
        next(refgen_file)
    for line in refgen_file:
        # skip missing BUSCOs
        if line.split()[1] == 'Missing':
            refgen_missing_count += 1
            continue
        (buscoid, status, contigid, rest_of_line) = line.split('\t', 3)
        # get contig name by removing ':number-number' from contigid
        contig_name = contigid.split(':')[0]
        # if contig_name in trinity_dict:
        #     refgen_dict[refgen_name] = f"{refgen_dict[contig_name]},{buscoid}"
        # else:
        #     refgen_dict[contig_name] = buscoid
        if buscoid in refgen_dict:
            refgen_dict[buscoid] = f"{refgen_dict[buscoid]},{contig_name}"
        else:
            refgen_dict[buscoid] = contig_name

In [ ]:
refprot_dict = {}
with open("/Users/danielli/Syncthing/UoB/Coursework/BIOLM0034_Research_Project/Project_proper/BUSCO_comparison/reference_genome_full_table.tsv") as refprot_file:
    # create missing counter; missing plus trinity_dict should equal 2510, the number of BUSCOs in the hemiptera_odb10.2019-11-20 database
    refprot_missing_count = 0
    # skip first 3 header lines
    for _ in range(3):
        next(refprot_file)
    for line in refprot_file:
        # skip missing BUSCOs
        if line.split()[1] == 'Missing':
            refprot_missing_count += 1
            continue
        (buscoid, status, contigid, rest_of_line) = line.split('\t', 3)
        # get contig name by removing ':number-number' from contigid
        contig_name = contigid.split(':')[0]
        # if contig_name in trinity_dict:
        #     refgen_dict[refgen_name] = f"{refgen_dict[contig_name]},{buscoid}"
        # else:
        #     refgen_dict[contig_name] = buscoid
        if buscoid in refprot_dict:
            refprot_dict[buscoid] = f"{refprot_dict[buscoid]},{contig_name}"
        else:
            refprot_dict[buscoid] = contig_name